In [1]:
!pip install -U /kaggle/input/faiss-cpu-173/faiss_cpu-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/faiss-cpu-173/faiss_cpu-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


FAISS (Facebook AI Similarity Search) is a library developed by Facebook AI Research, designed for efficient similarity search and clustering of dense vectors. we install it from dataset cuz in submission internet is turned off.

In [2]:
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers

The command copies the entire sentence-transformers directory (including all its contents) from the input directory (/kaggle/input/sentence-transformers-222) to the working directory (/kaggle/working). This is often done to make the files accessible for further processing or usage within the notebook, as the /kaggle/working directory is writable and intended for active work during the notebook session. Python library designed for computing dense vector representations for sentences, paragraphs, and images

In [3]:
!pip install -U /kaggle/working/sentence-transformers

Processing ./sentence-transformers
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126117 sha256=cd69a547974a2108be33f913e8032fdbca869b4f36ef7af0f448f1d103efc65a
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers


In [4]:
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl


In [5]:
import ctypes
libc = ctypes.CDLL("libc.so.6")

In [6]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
import torch

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

In [7]:
MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [8]:
if torch.cuda.is_available():
    device = 'cuda'
    print("CUDA is available. Using GPU.")
else:
    device = 'cpu'
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [29]:
WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH) #directory list of wikipedia dataset

In [30]:
trn = pd.read_csv("/kaggle/input/llm-science-exam-sciq-dataset/test_sciq.csv")
trn = trn.dropna()
trn

,id,prompt,D,E,A,C,B,answer


In [13]:
import pandas as pd

# Sample dataframes (trn1, trn2, ..., trn9) are assumed to be defined already

# Concatenate all dataframes
trn_merged = pd.concat([trn1, trn2, trn3, trn4, trn5, trn6, trn7, trn8, trn9], ignore_index=True, sort=False)

# Drop rows with any NaN values
trn_merged = trn_merged.dropna()
trn_merged.reset_index(inplace=True)

# Drop the existing 'id' column if it exists
if 'id' in trn_merged.columns:
    trn_merged = trn_merged.drop('id', axis=1)
    trn_merged = trn_merged.drop('index', axis=1)

trn_merged['id'] = np.arange(len(trn_merged))

In [14]:
trn = trn_merged
trn

,prompt,A,B,C,D,E,answer,id
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,0
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,1
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,2
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,3
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,4
...,...,...,...,...,...,...,...,...
6815,What are the ownership advantages of a proprie...,Ability to restrict use of the protocol and ch...,Ability to freely distribute implementations,Ability to enforce patent rights,Ability to make the protocol a trade secret,Ability to freely publish the protocol specifi...,A,6815
6816,How may proprietors enforce restrictions on a ...,By controlling intellectual property rights an...,By widely distributing implementations,By freely publishing the protocol specification,By allowing anyone to create an implementation,By making the protocol a trade secret,A,6816
6817,What is an example of a proprietary protocol?,TCP/IP,HTTP,Skype protocol,SMTP,FTP,C,6817
6818,What is the process of retrieving a protocol's...,Decompilation,Packet sniffing,Binary disassembly,Reverse engineering,Protocol extraction,D,6818


In [11]:
model = SentenceTransformer(MODEL, device=device)
model.max_seq_length = MAX_LENGTH
model = model.half()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [13]:
prompt_embeddings = model.encode(trn.prompt.values, batch_size=BATCH_SIZE, device=device, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True).half()
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [14]:
_ = gc.collect()

In [15]:
## Get the top 3 pages that are likely to contain the topic of interest
search_score, search_index = sentence_index.search(prompt_embeddings, 3)

In [16]:
## Save memory - delete sentence_index since it is no longer necessary
del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

1

In [17]:
df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet", columns=['id', 'file'])
df.head()

,id,file
0,49495844,a.parquet
1,3579086,a.parquet
2,62397582,a.parquet
3,15547032,a.parquet
4,8021609,a.parquet


In [18]:
## Get the article and associated file location using the index
wikipedia_file_data = []

for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    
    ## Get indices by score threshold
    #scr_idx = idx[np.where(scr <= 0.85)]
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

## Save memory - delete df since it is no longer necessary
del df
_ = gc.collect()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
wikipedia_file_data.head()

,id,prompt_id,file
0,10151726,582,a.parquet
1,1044136,491,a.parquet
2,106240,273,a.parquet
3,106240,358,a.parquet
4,106240,442,a.parquet


In [20]:
## Get the full text data
wiki_text_data = []

for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df = _df[_df['id'].isin(_id)]
    wiki_text_data.append(_df)
    _ = gc.collect()
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

  0%|          | 0/27 [00:00<?, ?it/s]

In [21]:
wiki_text_data.head()

,id,text
0,45502742,AC/DC is a pinball machine manufactured by Ste...
1,4362284,"AC/DC for General Exhibition, released only in..."
2,2225115,AC/DC Live is the second live album by Austral...
3,1665285,"In biology and ecology, abiotic components or ..."
4,571480,Absorbed dose is a dose quantity which is the ...


text_to_sentences_and_offsets function from the Bling Fire library (blingfire) to tokenize a document into sentences and obtain their character offsets within the original document

In [22]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)
    print(df.head())
    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    print(df.head())
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [23]:
## Parse documents into sentences
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)

  0%|          | 0/2519 [00:00<?, ?it/s]

  document_id                                               text      offset
0       10008  An electrode is an electrical conductor used t...  (0, 23681)
1    10013669  In evolutionary biology, function is the reaso...   (0, 6585)
2       10065  In chemistry, the empirical formula of a chemi...   (0, 3299)
3    10085128  The following is a list of the classes in each...   (0, 4436)
4    10085290  Lyme disease, or borreliosis, is caused by spi...  (0, 16135)


  0%|          | 0/2519 [00:00<?, ?it/s]

  document_id                                               text      offset
0       10008  An electrode is an electrical conductor used t...    (0, 154)
1       10008  Electrodes are essential parts of batteries th...  (155, 275)
2       10008  The electrophore, invented by Johan Wilcke, wa...  (276, 488)
3       10008  The first electrochemical battery made was dev...  (489, 610)
4       10008  Biography of Alessandro Volta – Stored Electri...  (611, 684)


In [24]:
## Get embeddings of the wiki text data
wiki_data_embeddings = model.encode(processed_wiki_text_data.text, batch_size=BATCH_SIZE, device=device, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True).half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/9695 [00:00<?, ?it/s]

In [25]:
wiki_data_embeddings

array([[-0.0399   ,  0.0648   ,  0.04544  , ...,  0.05402  ,  0.036    ,
         0.03165  ],
       [-0.0437   ,  0.07904  ,  0.04007  , ...,  0.05795  ,  0.0871   ,
         0.00616  ],
       [-0.03647  ,  0.133    ,  0.0257   , ...,  0.0654   ,  0.08185  ,
         0.03723  ],
       ...,
       [ 0.0365   ,  0.01442  ,  0.063    , ...,  0.0003355, -0.1098   ,
        -0.04132  ],
       [-0.00284  ,  0.01915  ,  0.02888  , ..., -0.02487  , -0.05255  ,
        -0.04794  ],
       [ 0.02475  , -0.008934 ,  0.0827   , ...,  0.01822  , -0.07684  ,
        -0.01229  ]], dtype=float16)

In [26]:
_ = gc.collect()

In [27]:
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)

## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

TypeError: sequence item 0: expected str instance, float found

In [ ]:
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=device, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True).half()
question_embeddings = question_embeddings.detach().cpu().numpy()

In [ ]:
## Parameter to determine how many relevant sentences to include
NUM_SENTENCES_INCLUDE = 3

## List containing Question, Choices, Context
prompt_contexts = []

## List containing just Context
contexts = []

for r in trn.itertuples():
    prompt_context = ""

    prompt_id = r.id

    prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values)].index.values
    prompt_context += "Question: " + trn.prompt.iloc[prompt_id] + "\n"

    prompt_context += "Choices:\n"
    prompt_context += "(A) " + trn.A.iloc[prompt_id] + "\n"
    prompt_context += "(B) " + trn.B.iloc[prompt_id] + "\n"
    prompt_context += "(C) " + trn.C.iloc[prompt_id] + "\n"
    prompt_context += "(D) " + trn.D.iloc[prompt_id] + "\n"
    prompt_context += "(E) " + trn.E.iloc[prompt_id] + "\n"

    if prompt_indices.shape[0] > 0:
        prompt_context += "Context:\n"
        ## Per Prompt Index
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        
        prompt_index.add(wiki_data_embeddings[prompt_indices])
        

        context = ""
        
        ## Get the top matches
        ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
            ## Threshold on the score
            if _s < 2:
                context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + "\n"
        prompt_context += context
        
        
    contexts.append(context)
    prompt_contexts.append(prompt_context)

In [ ]:
trn['context'] = contexts

In [ ]:
trn = trn.drop('answer_all', axis=1)
trn = trn.drop('prompt_answer_stem', axis=1)
trn

In [ ]:
trn.to_csv("./train_context.csv", index=False)

In [ ]:
for i, p in enumerate(prompt_contexts[:10]):
    print(f"Question {i}")
    print(p)
    print()